In [1]:
#files
import sys
from os import listdir
sys.path.append('../data')
sys.path.append('../Librerias')
from os.path import isfile, join


#own libs
import utils
import caract as dc
import dataset as ds


#data and visualization
import copy
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go


#torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset


sns.set_style("darkgrid")

## **data**

In [2]:
fpath = "/Users/granosftp/Documents/GitHub/Tesis/data/high/"
#fpath =  "/Users/consu/OneDrive/Documentos/GitHub/Tesis/data/low/"
onlyfiles = [f for f in listdir(fpath) if isfile(join(fpath, f))]

cutoff = [8/1000, 11/1000]

#load data
frames = []
for fname in onlyfiles:
    if fname.endswith(".mat"):
        set =  ds.MatFileToDataFrame(fpath, fname)
        df = set.get_dataframe(cutoff)
        frames.append(df)
        print(fname)

frames = frames[:2]

high8_090124.mat
high5_100124.mat
high10_090124.mat
high10_100124.mat
high8_100124.mat
high5_090124.mat
high2_100124.mat
high7_100124.mat
high12_090124.mat
high7_090124.mat
high9_090124.mat
high9_100124.mat
high4_090124.mat
high1_090124.mat
high1_100124.mat
high6_100124.mat
high3_090124.mat
high3_100124.mat


In [3]:
defectos_frames = []
taus = []
for i, df in enumerate(frames):
    aux = dc.CaractDefect(df)
    defectos_frames.append(aux)
    taus.append(aux.get_tau()[1])
    sys.stdout.write(str(i) + '\r')

In [4]:
hilbert_frames = []
for i, df in enumerate(defectos_frames):
    aux, _ = df.get_hilbert()
    hilbert_frames.append(aux)
    sys.stdout.write(str(i) + '\r')

In [5]:
# all frames
frames_all = []

for i in range(len(frames)):
    frames_all.append(pd.merge(frames[i], hilbert_frames[i], on='Hilbert Transform', how='outer'))

frames_all[0]

,Original Signal,Signal - Mean,Filtered Signal,Hilbert Transform,Amplitude,Instantaneous Phase,Gradient Phase
0,0.892414,0.286351,0.035339,0.035339-0.029502j,0.046035,-0.695619,-0.031476
1,0.905895,0.299832,0.033637,0.033637-0.002503j,0.033730,-0.074284,-0.002016
2,0.917075,0.311011,0.031906,0.031906-0.000839j,0.031917,-0.026296,0.025766
3,0.930556,0.324492,0.030149,0.030149+0.008867j,0.031426,0.286053,0.050336
4,0.939105,0.333041,0.028367,0.028367+0.010115j,0.030117,0.342504,0.070494
...,...,...,...,...,...,...,...
599995,0.371251,-0.234812,-0.003465,-0.003465-0.005847j,0.006797,18853.733329,0.147866
599996,0.348564,-0.257500,-0.003298,-0.003298-0.004618j,0.005674,18853.648171,0.158010
599997,0.344618,-0.261446,-0.003127,-0.003127-0.011439j,0.011859,18854.001503,0.167368
599998,0.325218,-0.280845,-0.002952,-0.002952-0.009812j,0.010247,18853.976039,0.175975


### **series for training**

In [6]:
frames_all[0].to_csv('frame0.csv', index=False)

In [7]:
serie = frames_all[0][['Filtered Signal','Amplitude', 'Gradient Phase']].values
serie

array([[ 0.0353394 ,  0.04603539, -0.03147597],
       [ 0.03363706,  0.03373008, -0.00201604],
       [ 0.03190641,  0.03191744,  0.02576591],
       ...,
       [-0.00312655,  0.01185856,  0.16736794],
       [-0.00295232,  0.01024654,  0.17597486],
       [-0.00277543,  0.0324514 ,  0.18386933]])

In [8]:
def load_data(Serie,size_train = 0.8, size_val = 0.3, batch_size=32):
    trainval_size = int(size_train*len(serie))
    val_size =  int(size_val*trainval_size)
    train_size = trainval_size - val_size

    X, y = utils.create_sequences(serie, 1000, 100, 0)
    
    #split de data
    X_train, y_train = X[:train_size], y[:train_size]
    X_val, y_val = X[train_size:trainval_size], y[train_size:trainval_size]
    X_test, y_test = X[trainval_size:], y[trainval_size:]

    x_train = torch.from_numpy(X_train).float()
    x_val = torch.from_numpy(X_val).float()
    x_test = torch.from_numpy(X_test).float()

    y_train = torch.from_numpy(y_train).float()
    y_val = torch.from_numpy(y_val).float()
    y_test = torch.from_numpy(y_test).float()
    
    #create dataset tensor
    train_dataset = TensorDataset(x_train, y_train)
    val_dataset = TensorDataset(x_val, y_val)
    test_dataset = TensorDataset(x_test, y_test)

    #create dataloader
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

    return train_dataloader, val_dataloader, test_dataloader

In [9]:
# train_dataloader, val_dataloader, test_dataloader = load_data(serie)

In [10]:
# Preprocess the historical data
seq_length = 1000
X, y = utils.create_sequences(serie, seq_length, 100, 0)

# Split the data into training and testing sets
train_size = int(len(y) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Convert to PyTorch tensors
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

## **modelo 1**

In [11]:
class LSTMModel(nn.Module):

    def __init__(self, input_size, hidden_size, output_size, num_layers = 1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        #self.sigmoid = nn.Sigmoid(hidden_size)



    def forward(self, x):
        
        h0 =  torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 =  torch.zeros(self.num_layers, x.size(0), self.hidden_size)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out

In [12]:
input_size = 3
hidden_size = 50
num_layers = 1
output_size = 2

model = LSTMModel(input_size, hidden_size, output_size, num_layers)

In [13]:
X_train.shape

torch.Size([479120, 1000, 3])

In [14]:
# Set training parameters
learning_rate = 0.01
num_epochs = 100

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    outputs = model(X_train)#.unsqueeze(-1)).squeeze()  # Add .squeeze() here
    optimizer.zero_grad()
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

    #if (epoch + 1) % 10 == 0:
        

: 

In [ ]:
'''# entrenamiento
lr = 0.01
epochs = 100

# loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(epochs):
    
    optimizer.zero_grad()
    
    for X_batch, y_batch in train_dataloader:
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    
    # validation step
    model.eval()
    with torch.no_grad():
        for X_val_batch, y_val_batch in val_dataloader:
            val_pred = model(X_val_batch)
            val_loss = criterion(val_pred, y_val_batch)
            print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}') 

    model.train()
    '''

/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 100, 2])) that is different to the input size (torch.Size([32, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (32) must match the size of tensor b (100) at non-singleton dimension 1